In [ ]:
import numpy as np
import pandas as pd
import sys, os, re, time, shutil, yaml
import defusedxml.ElementTree as ET
from collections import defaultdict
from pyfitparquet import transformer, loadconfig

In [ ]:
pyfitparq = transformer.PyFitParquet()
pyfitparq.source_to_parquet('./bogusfile.fit', parquet_dir='.')

In [ ]:
DATA_DIR = '../test/fixtures'
PARQUET_DIR = os.path.join(DATA_DIR, 'parquet')

# Serializes all files in DATA_DIR to parquet
pyfitparq = transformer.PyFitParquet()
pyfitparq.data_to_parquet(DATA_DIR)

In [ ]:
parquet_files = []
for file in sorted(os.listdir(DATA_DIR)):
    matchfit = re.match('(\w+).(fit|FIT)$', file)
    matchtcx = re.match('(\w+).(tcx|TCX)$', file)            
    if matchfit: parquet_files.append(f"{PARQUET_DIR}/{matchfit.group(1)}.parquet")            
    elif matchtcx: parquet_files.append(f"{PARQUET_DIR}/{matchtcx.group(1)}.parquet")
#}

parquet_files

In [ ]:
for pfile in parquet_files:
    df = pd.read_parquet(pfile, engine='pyarrow')
    pseries = df[df["field_name"] == "power"]["value_float"]
    if len(pseries) > 0: print(round(pseries.mean()), end=',')
        
print()
for pfile in parquet_files:
    df = pd.read_parquet(pfile, engine='pyarrow')
    pseries = df[df["field_name"] == "power"]["value_float"]
    if len(pseries) > 0: print(f"\'{pfile.split('/')[-1]}\'", end=', ')

In [ ]:
for pfile in parquet_files:
    df = pd.read_parquet(pfile, engine='pyarrow')
    hrseries = df[df["field_name"] == "heart_rate"]["value_float"]
    if len(hrseries) > 0: print((int(hrseries.min()), int(hrseries.max()), round(hrseries.mean())), end=',')
        
print()
for pfile in parquet_files:
    df = pd.read_parquet(pfile, engine='pyarrow')
    hrseries = df[df["field_name"] == "heart_rate"]["value_float"]
    if len(hrseries) > 0: print(f"\'{pfile.split('/')[-1]}\'", end=', ')

In [ ]:
for pfile in parquet_files:
    df = pd.read_parquet(pfile, engine='pyarrow')
    dseries = df[df["field_name"] == "distance"]["value_float"]
    if len(dseries) > 0: print(round(dseries.sum()), end=',')

print()
for pfile in parquet_files:
    df = pd.read_parquet(pfile, engine='pyarrow')
    dseries = df[df["field_name"] == "distance"]["value_float"]
    if len(dseries) > 0: print(f"\'{pfile.split('/')[-1]}\'", end=', ')

In [ ]:
loadconfig.populate_config() 
loadconfig.config_dir

# Input: base install, CONDA_PREFIX set PYFIT_CONFIG_DIR not set, no local files
# Expected result: loadconfig.config_dir == site-package dir of CONDA env install

In [ ]:
conda_prefix = None
if 'CONDA_PREFIX' in os.environ:
    conda_prefix = os.environ['CONDA_PREFIX']
    del os.environ['CONDA_PREFIX']

try: loadconfig.populate_config()
except AssertionError as err: print(f'Caught assert: {err}')
print(loadconfig.config_dir)

# Input: CONDA_PREFIX not set, PYFIT_CONFIG_DIR not set, no local files
# Expected result: assertion error, can't find config_files

In [ ]:
os.environ['PYFIT_CONFIG_DIR'] = os.getcwd()
try: loadconfig.populate_config()
except AssertionError as err: print(f'Caught assert: {err}')
print(loadconfig.config_dir)

# Input: CONDA_PREFIX not set, PYFIT_CONFIG_DIR is set, no local files
# Expected result: assertion error, can't find config_files

In [ ]:
os.environ['CONDA_PREFIX'] = conda_prefix
loadconfig.populate_config() 
loadconfig.config_dir

# Input: CONDA_PREFIX set, PYFIT_CONFIG_DIR set, no local files
# Expected result: loadconfig.config_dir == PYFIT_CONFIG_DIR, files copied in

In [ ]:
if 'CONDA_PREFIX' in os.environ:
    conda_prefix = os.environ['CONDA_PREFIX']
    del os.environ['CONDA_PREFIX']

loadconfig.populate_config()
loadconfig.config_dir

# Input: CONDA_PREFIX not set, PYFIT_CONFIG_DIR set, local files exist
# Expected result: loadconfig.config_dir == PYFIT_CONFIG_DIR